<a href="https://colab.research.google.com/github/ManasviAtGitHub/Algorithms-for-Optimization/blob/main/Gradient_Descent_in_Julia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" /> _Colab Notebook Template_

## Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Harware accelerator_ = _GPU_.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). This takes a couple of minutes.
4. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

Original Source : https://www.jihongzhang.org/post/gradient-descent-via-julia/

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.6.0" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools Plots"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Success! Please reload this page and jump to the next section."
fi

Installing Julia 1.6.0 on the current Colab Runtime...
2021-11-19 16:18:43 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.6/julia-1.6.0-linux-x86_64.tar.gz [112838927/112838927] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package BenchmarkTools...
Installing Julia package Plots...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.6

Success! Please reload this page and jump to the next section.


# Need Help?

* Learning: https://julialang.org/learning/
* Documentation: https://docs.julialang.org/
* Questions & Discussions:
  * https://discourse.julialang.org/
  * http://julialang.slack.com/
  * https://stackoverflow.com/questions/tagged/julia

If you ever ask for help or file an issue about Julia, you should generally provide the output of `versioninfo()`.

Add new code cells by clicking the `+ Code` button (or _Insert_ > _Code cell_).

Have fun!

<img src="https://raw.githubusercontent.com/JuliaLang/julia-logo-graphics/master/images/julia-logo-mask.png" height="100" />

In [ ]:
]add RDatasets,DataFrames

    Updating registry at `~/.julia/registries/General`
   Resolving package versions...
   Installed Mocking ──────────── v0.7.3
   Installed Crayons ──────────── v4.0.4
   Installed InlineStrings ────── v1.0.1
   Installed ExprTools ────────── v0.1.6
   Installed RData ────────────── v0.8.3
   Installed SentinelArrays ───── v1.3.8
   Installed CodecZlib ────────── v0.7.0
   Installed PooledArrays ─────── v1.3.0
   Installed DataFrames ───────── v1.2.2
   Installed FileIO ───────────── v1.11.2
   Installed FilePathsBase ────── v0.9.16
   Installed WeakRefStrings ───── v1.4.1
   Installed TranscodingStreams ─ v0.9.6
   Installed InvertedIndices ──── v1.1.0
   Installed CategoricalArrays ── v0.10.2
   Installed TimeZones ────────── v1.6.2
   Installed CSV ──────────────── v0.9.11
   Installed PrettyTables ─────── v1.2.3
   Installed RDatasets ────────── v0.7.6
    Updating `~/.julia/environments/v1.6/Project.toml`
  [a93c6f00] + DataFrames v1.2.2
  [ce6b1742] + RDatasets v0.7.6
    Updat

In [ ]:
using RDatasets
using DataFrames

In [ ]:
mtcars = dataset("datasets", "mtcars")

,Model,MPG,Cyl,Disp,HP,DRat,WT,QSec
,String3…,Float64,Int64,Float64,Int64,Float64,Float64,Float64
1,Mazda RX4,21.0,6,160.0,110,3.9,2.62,16.46
2,Mazda RX4 Wag,21.0,6,160.0,110,3.9,2.875,17.02
3,Datsun 710,22.8,4,108.0,93,3.85,2.32,18.61
4,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44
5,Hornet Sportabout,18.7,8,360.0,175,3.15,3.44,17.02
6,Valiant,18.1,6,225.0,105,2.76,3.46,20.22
7,Duster 360,14.3,8,360.0,245,3.21,3.57,15.84
8,Merc 240D,24.4,4,146.7,62,3.69,3.19,20.0
9,Merc 230,22.8,4,140.8,95,3.92,3.15,22.9


In [ ]:
function gradientDesc(x, y, learn_rate, conv_threshold, n, max_iter)
    β = rand(Float64, 1)[1]
    α = rand(Float64, 1)[1]
    ŷ = α .+ β .* x
    MSE = sum((y .- ŷ).^2)/n
    converged = false
    iterations = 0

    while converged == false
        # Implement the gradient descent algorithm
        β_new = β - learn_rate*((1/n)*(sum((ŷ .- y) .* x)))
        α_new = α - learn_rate*((1/n)*(sum(ŷ .- y)))
        α = α_new
        β = β_new
        ŷ = β.*x .+ α
        MSE_new = sum((y.-ŷ).^2)/n
        # decide on whether it is converged or not
        if (MSE - MSE_new) <= conv_threshold
            converged = true
            println("Optimal intercept: $α; Optimal slope: $β")
        end
        iterations += 1
        if iterations > max_iter
            converged = true
            println("Optimal intercept: $α; Optimal slope: $β")
        end
    end
end

gradientDesc (generic function with 1 method)

In [ ]:
gradientDesc(mtcars[:,:Disp], mtcars[:,:MPG], 0.0000293, 0.001, 32, 2500000)

Optimal intercept: 29.599851535611556; Optimal slope: -0.04121510905370332


In [ ]:
]add GLM

   Resolving package versions...
   Installed OpenSpecFun_jll ── v0.5.5+0
   Installed Rmath ──────────── v0.7.0
   Installed FillArrays ─────── v0.12.7
   Installed StatsFuns ──────── v0.9.13
   Installed QuadGK ─────────── v2.4.2
   Installed StatsModels ────── v0.6.28
   Installed GLM ────────────── v1.5.1
   Installed Distributions ──── v0.25.31
   Installed ShiftedArrays ──── v1.0.0
   Installed Rmath_jll ──────── v0.3.0+0
   Installed DensityInterface ─ v0.4.0
   Installed PDMats ─────────── v0.11.5
   Installed SpecialFunctions ─ v1.8.1
    Updating `~/.julia/environments/v1.6/Project.toml`
  [38e38edf] + GLM v1.5.1
    Updating `~/.julia/environments/v1.6/Manifest.toml`
  [b429d917] + DensityInterface v0.4.0
  [31c24e10] + Distributions v0.25.31
  [1a297f60] + FillArrays v0.12.7
  [38e38edf] + GLM v1.5.1
  [90014a1f] + PDMats v0.11.5
  [1fd47b50] + QuadGK v2.4.2
  [79098fc4] + Rmath v0.7.0
  [1277b4bf] + ShiftedArrays v1.0.0
  [276daf66] + SpecialFunctions v1.8.1
  [4c63d2b9] +

In [ ]:
using GLM
linearRegressor = lm(@formula(MPG ~ Disp), mtcars)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

MPG ~ 1 + Disp

Coefficients:
───────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error      t  Pr(>|t|)  Lower 95%   Upper 95%
───────────────────────────────────────────────────────────────────────────
(Intercept)  29.5999     1.22972     24.07    <1e-20  27.0884    32.1113
Disp         -0.0412151  0.00471183  -8.75    <1e-09  -0.050838  -0.0315923
───────────────────────────────────────────────────────────────────────────